In [1]:
from tsai.models import InceptionTime, TST, ResNet
# from tsai.data.core import get_UCR_data
# from tsai.learner import TSClassifier
import os
import numpy as np
import sys
print(sys.executable)
import sys
sys.path.insert(1, '../src/')
from config import raw_data_path, univariate_data_path, processed_data_path
from preprocessing_modules import create_time_windows_with_labels
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import roc_auc_score, average_precision_score
import matplotlib.pyplot as plt
from collections import Counter
import torch.nn as nn
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from sklearn.utils.class_weight import compute_class_weight
import wandb
wandb.login(key="5f15eb7efc1e0e939ccc83345338a0b8c24e2fbc")
from fastai.optimizer import SGD
from fastai.callback.tracker import SaveModelCallback, EarlyStoppingCallback


/home/nwertheim/miniconda3/bin/python


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/nwertheim/.netrc
wandb: Currently logged in as: noorwertheim to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [2]:
num_epochs = 10

In [3]:
'''MY DATA'''
train_dir = os.path.join(univariate_data_path, "target_univariate_no_PCA_train_2_80.npy")
train_data = np.load(train_dir, allow_pickle=True)
# train_data = pd.DataFrame(train_data)
# test_dir = os.path.join(univariate_data_path, "target_univariate_no_PCA_test_2.npy")
# test_data = np.load(test_dir, allow_pickle=True)
# test_data = pd.DataFrame(test_data)
print(len(train_data))
# print(len(test_data))
print(train_data[0])
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.utils.class_weight import compute_class_weight

from collections import Counter

# Example dataset with windows and labels
train_windows, train_labels = create_time_windows_with_labels(train_data)
# test_windows, test_labels = create_time_windows_with_labels(test_data)

# Count label distribution
train_label_counts = Counter(train_labels)
# test_label_counts = Counter(test_labels)

# Print the counts
print("Train label distribution:")
print(f"  Term (0): {train_label_counts[0]}")
print(f"  Preterm (1): {train_label_counts[1]}")

# print("\nTest label distribution:")
# print(f"  Term (0): {test_label_counts[0]}")
# print(f"  Preterm (1): {test_label_counts[1]}")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Convert to tensors
train_windows_tensor = torch.tensor(train_windows, dtype=torch.float32)
train_labels_tensor = torch.tensor(train_labels, dtype=torch.float32)
# test_windows_tensor = torch.tensor(test_windows, dtype=torch.float32)
# test_labels_tensor = torch.tensor(test_labels, dtype=torch.float32)

train_windows_tensor = train_windows_tensor.unsqueeze(1)  # (1071, 1, 12000)
# test_windows_tensor = test_windows_tensor.unsqueeze(1)    # (899, 1, 12000)

train_labels_tensor = train_labels_tensor.long()
# test_labels_tensor = test_labels_tensor.long()

595
{'record_name': 'Hopper-2022_02_08_15_07_31-0000010181-0011-chan3', 'signal': array([[14.23043527],
       [24.57069717],
       [17.99612535],
       ...,
       [-1.21857878],
       [ 0.82743019],
       [-2.60873858]]), 'fs': 20, 'preterm': 1}
windows [[ 1.42304353e+01  2.45706972e+01  1.79961254e+01 ... -4.22795156e-01
  -4.24569100e-01 -4.19660561e-01]
 [-4.09576709e-01 -3.93029615e-01 -3.71712658e-01 ...  8.95526968e-02
   6.79439064e-02  4.53160159e-02]
 [ 2.14448065e-02 -2.85958066e-03 -2.77785974e-02 ...  3.43722578e-01
   3.17636136e-01  2.88491584e-01]
 ...
 [-3.28359489e-01 -3.42673111e-01 -3.52797715e-01 ...  8.64233555e-02
  -3.15361868e-02 -1.49023389e-01]
 [-2.64885369e-01 -3.77907608e-01 -4.86977459e-01 ... -7.73425655e-01
  -7.59971006e-01 -7.36558185e-01]
 [-7.02548059e-01 -6.57195623e-01 -6.00161840e-01 ... -4.41643237e-01
  -5.73274804e-01 -6.94947761e-01]]
labels [1 1 1 ... 1 1 1]
Train label distribution:
  Term (0): 2623
  Preterm (1): 5248


In [4]:
import torch
import optuna
import numpy as np
from tsai.models.ResNet import ResNet
from fastai.learner import Learner
from fastai.data.core import DataLoaders
from fastai.losses import CrossEntropyLossFlat
from sklearn.model_selection import KFold
from torch.utils.data import TensorDataset, Subset
from sklearn.metrics import average_precision_score
from fastai.metrics import Metric
from fastai.callback.wandb import WandbCallback
from fastai.optimizer import SGD, Adam
# Create TensorDataset from your data
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

dataset = TensorDataset(train_windows_tensor, train_labels_tensor)

# ===== Compute class weights =====
train_labels_np = train_labels_tensor.numpy()
class_weights = compute_class_weight(class_weight="balanced", classes=np.unique(train_labels_np), y=train_labels_np)
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32).to(device)

print(len(train_windows_tensor))
# Custom Average Precision Metric
class AveragePrecision(Metric):
    def __init__(self):
        self.pred = []
        self.target = []

    def reset(self):
        self.pred, self.target = [], []

    def accumulate(self, learn):
        preds = learn.pred.detach().softmax(dim=-1)[:, 1]  # Get probabilities for class 1
        targs = learn.y.detach()
        self.pred.append(preds.cpu())
        self.target.append(targs.cpu())

    @property
    def value(self):
        preds = torch.cat(self.pred).numpy()
        targs = torch.cat(self.target).numpy()
        return average_precision_score(targs, preds)

    @property
    def name(self): return "avg_precision"

def objective(trial):
    print('Trial:', trial)
    print(f"Starting trial {trial.number}")

    learning_rate = trial.suggest_loguniform('lr', 1e-7, 1e-6)
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64])
    optimizer_name = trial.suggest_categorical('optimizer', ['SGD', 'Adam'])

    # Map optimizer name to fastai optimizer function
    opt_func_map = {
    'SGD': SGD,
    'Adam': Adam,
    }

    opt_func = opt_func_map[optimizer_name]

    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    avg_precisions = []

    for train_idx, val_idx in kf.split(np.arange(len(dataset))):
        train_dataset = Subset(dataset, train_idx)
        val_dataset = Subset(dataset, val_idx)

        dls = DataLoaders.from_dsets(
            train_dataset,
            val_dataset,
            bs=batch_size,
            shuffle=True,
            num_workers=0
        )

        model = ResNet(c_in=1, c_out=2).to(device)

        loss_func = torch.nn.CrossEntropyLoss(weight=class_weights_tensor)

        learn = Learner(
            dls, model,
            loss_func=loss_func,
            opt_func=opt_func,
            metrics=AveragePrecision(),
            cbs=[
                EarlyStoppingCallback(monitor='valid_loss', patience=3),
            ]
        )

        learn.fit_one_cycle(num_epochs, learning_rate)
        ap = learn.validate()[1]  
        avg_precisions.append(ap)
        

    return np.mean(avg_precisions)

# Run Optuna study
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=25)

print("Best trial:")
print(study.best_trial)


# Save the study
import joblib
joblib.dump(study, "ResNet_tune_80.pkl")

/home/nwertheim/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-05-15 22:57:55,170] A new study created in memory with name: no-name-dc3eaf4e-4b00-4202-8397-3f9647f658af


7871
Trial: <optuna.trial._trial.Trial object at 0x1464706d58e0>
Starting trial 0


epoch,train_loss,valid_loss,avg_precision,time
0,0.707901,0.709808,0.620365,00:07
1,0.703586,0.702904,0.622088,00:06
2,0.693340,0.697027,0.627663,00:06
3,0.691981,0.694765,0.718600,00:06
4,0.690306,0.692504,0.735758,00:06
5,0.688797,0.692194,0.738948,00:06
6,0.687991,0.692347,0.739503,00:06
7,0.690244,0.692561,0.735915,00:06
8,0.688751,0.692347,0.737907,00:06


No improvement since epoch 5: early stopping


epoch,train_loss,valid_loss,avg_precision,time
0,0.708366,0.715773,0.641380,00:06
1,0.703717,0.705196,0.641346,00:06
2,0.698036,0.696059,0.645228,00:06
3,0.693407,0.692336,0.724727,00:06
4,0.693782,0.689730,0.769736,00:06
5,0.692810,0.688564,0.778750,00:06
6,0.691534,0.687968,0.781176,00:06
7,0.691937,0.688624,0.762436,00:06
8,0.692375,0.687374,0.780636,00:06
9,0.690360,0.688401,0.763516,00:06


...

No improvement since epoch 2: early stopping


epoch,train_loss,valid_loss,avg_precision,time
0,0.696289,0.695686,0.732064,00:07
1,0.692470,0.692317,0.727310,00:07
2,0.689429,0.689121,0.772063,00:07
3,0.690490,0.689382,0.741331,00:07
4,0.687450,0.687600,0.777123,00:07
5,0.690567,0.687600,0.735048,00:07
6,0.688888,0.687037,0.737728,00:07
7,0.688839,0.686305,0.734690,00:07
8,0.688997,0.685971,0.736562,00:07
9,0.685114,0.686694,0.737286,00:07


[I 2025-05-16 01:07:00,205] Trial 24 finished with value: 0.7471216881497057 and parameters: {'lr': 7.754767698719833e-07, 'batch_size': 16, 'optimizer': 'Adam'}. Best is trial 21 with value: 0.75965830547512.


Best trial:
FrozenTrial(number=21, state=1, values=[0.75965830547512], datetime_start=datetime.datetime(2025, 5, 16, 0, 45, 26, 331182), datetime_complete=datetime.datetime(2025, 5, 16, 0, 51, 0, 536162), params={'lr': 9.852044440006024e-07, 'batch_size': 32, 'optimizer': 'Adam'}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'lr': FloatDistribution(high=1e-06, log=True, low=1e-07, step=None), 'batch_size': CategoricalDistribution(choices=(16, 32, 64)), 'optimizer': CategoricalDistribution(choices=('SGD', 'Adam'))}, trial_id=21, value=None)


['ResNet_tune_80.pkl']

In [5]:


print(study.best_trial)

FrozenTrial(number=21, state=1, values=[0.75965830547512], datetime_start=datetime.datetime(2025, 5, 16, 0, 45, 26, 331182), datetime_complete=datetime.datetime(2025, 5, 16, 0, 51, 0, 536162), params={'lr': 9.852044440006024e-07, 'batch_size': 32, 'optimizer': 'Adam'}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'lr': FloatDistribution(high=1e-06, log=True, low=1e-07, step=None), 'batch_size': CategoricalDistribution(choices=(16, 32, 64)), 'optimizer': CategoricalDistribution(choices=('SGD', 'Adam'))}, trial_id=21, value=None)


In [6]:
# import torch
# import optuna
# import numpy as np
# from tsai.models.ResNet import ResNet
# from fastai.learner import Learner
# from fastai.data.core import DataLoaders
# from fastai.metrics import accuracy
# from sklearn.model_selection import KFold
# from torch.utils.data import TensorDataset, Subset
# from fastai.callback.wandb import WandbCallback
# from fastai.data.core import DataLoaders
# from fastai.learner import Learner
# from fastai.metrics import accuracy
# from fastai.losses import CrossEntropyLossFlat
# from sklearn.model_selection import KFold
# from torch.utils.data import Subset
# from tsai.models.ResNet import ResNet

# # Create TensorDataset from your data
# dataset = TensorDataset(train_windows_tensor, train_labels_tensor)

# def objective(trial):
#     print('Trial: ', trial)
#     learning_rate = trial.suggest_float("lr", 1e-6, 1e-3, log=True)
#     batch_size = trial.suggest_categorical("batch_size", [16, 32, 64])

#     kf = KFold(n_splits=5, shuffle=True, random_state=42)
#     accs = []

#     for train_idx, val_idx in kf.split(np.arange(len(dataset))):
#         train_dataset = Subset(dataset, train_idx)
#         val_dataset = Subset(dataset, val_idx)

#         dls = DataLoaders.from_dsets(
#             train_dataset,
#             val_dataset,
#             bs=batch_size,
#             shuffle=True,
#             num_workers=0  # for compatibility and reproducibility
#         )
#         model = ResNet(c_in=1, c_out=2)
#         learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(), metrics=accuracy)

#         learn.fit_one_cycle(5, lr_max=learning_rate)
#         acc = learn.validate()[1].item()
#         accs.append(acc)

#     return np.mean(accs)

# study = optuna.create_study(direction="maximize")
# study.optimize(objective, n_trials=25)

# print("Best trial:")
# print(study.best_trial)

# # Save the study if you want
# import joblib
# joblib.dump(study, "resnet_optuna_study.pkl")
